Countries Analysis

In [ ]:
import pandas as pd
import requests
import json
import awswrangler as wr

In [ ]:
# Fecthing the dat from api

api_url = 'https://restcountries.com/v3.1/all'
response = requests.get(api_url)
data = response.json()

In [ ]:
# normalizing the JSON data 

df = pd.json_normalize(data)
df.info()

In [ ]:
# Dropping the unecessary columns 

# Multi column dropping
df_cols_to_drop = [col for col in df.columns if 'demonyms' in col or 'translations' in col or 'currencies' in col or 'coatOfArms' in col or 'postalCode' in col or 'name.nativeName' in col or 'name.tsn' in col or 'idd' in col or 'altSpellings' in col or 'gini' in col  or 'languages' in col]
df.drop(columns=df_cols_to_drop, inplace=True)

# Single column dropping 
df.drop(columns='flag', inplace = True)


In [ ]:
# No concatenating of outputs
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_item', None)

In [ ]:
# Single column cleaning

df['continents'] = df["continents"].astype(str).str.strip('[]\'')
df['capital'] = df["capital"].apply(str).str.strip('[]\'\"')

# Apply to the whole of dataframe
for col in df.columns:
    df[col] = df[col].apply(lambda col: str(col).strip('[]\'\"'))

df['borders'] = df['borders'].astype(str).str.replace("'","",regex= False)

df.head(1)

In [ ]:
# Splitting of column in seperate for expansion of columns

df[['Latitude', 'Longitude']] = df['latlng'].str.split(',', n = 1, expand = True)
df[['Capital latitude', 'Capital Longitude']] = df['capitalInfo.latlng'].str.split(',', n = 1, expand = True)


columnsDropped = ['latlng', 'capitalInfo.latlng']
df.drop(columns = 'latlng', inplace = True)
df.head(1)

In [ ]:
# Capital first letter in startOfWeek Column

def weekConversion(days):
    weekday = ""
    for i in range(0,len(days)):
        if i == 0:
            i = str(days[i]).upper()
            weekday += i
        else:
            weekday += days[i]
    return weekday
    
df['startOfWeek'] = df['startOfWeek'].apply(weekConversion)

In [ ]:
df['status'].value_counts()

In [ ]:
df_cleaned_data = df.copy()